In [ ]:
! pip install pymupdf pdfplumber jieba

In [ ]:
import fitz  # PyMuPDF

def read_pdf_text(pdf_path, page_num):
    doc = fitz.open(pdf_path)
    text = ""
    '''
    for page in doc:
        text += page.get_text("text") + "\n"
    return text
    '''
    return doc[page_num].get_text("text")
# 測試讀取 PDF
pdf_file = "./pdf_file/中央論文1.pdf"  # 請確保這個 PDF 檔案存在
print(read_pdf_text(pdf_file, 25))


## 嘗試抓文字位置

In [ ]:
import jieba

def extract_text_with_positions(pdf_path):
    doc = fitz.open(pdf_path)
    text_positions = []
    
    
    for page_num, page in enumerate(doc):
        blocks = page.get_text("blocks")  # 取得所有單字及其位置
        for block in blocks:
            x0, y0, x1, y1, text, *_ = block  # 取得單字位置與內容
            #words = list(jieba.cut(text))
            text_positions.append((page_num, text, (x0, y0, x1, y1)))
    
    
    return text_positions

text_positions = extract_text_with_positions(pdf_file)
for pos in text_positions:
    print(pos) 

In [ ]:
def extract_text_test(pdf_path, page_num):
    doc = fitz.open(pdf_path)
    text_positions = []
    page = doc[page_num]
    blocks = page.get_text("blocks")  # 取得所有單字及其位置
    for block in blocks:
        x0, y0, x1, y1, text, *_ = block  # 取得單字位置與內容
        text_positions.append((page_num, text, (x0, y0, x1, y1)))
    return text_positions

text_positions = extract_text_test(pdf_file, 18)
for pos in text_positions:
    print(pos)  
    

In [ ]:
text_positions = extract_text_test(pdf_file, 19)
for pos in text_positions:
    print(pos)  

# 抓圖片測試

In [ ]:
import os

def extract_images_from_page(pdf_path, page_number, output_folder="images"):
    # 打開 PDF 文件
    doc = fitz.open(pdf_path)
    page = doc[page_number]  # 取得指定頁面
    
    # 建立輸出資料夾
    os.makedirs(output_folder, exist_ok=True)

    image_count = 0
    for img_index, img in enumerate(page.get_images(full=True)):
        xref = img[0]  # 取得圖片的 xref
        base_image = doc.extract_image(xref)  # 提取圖片數據
        image_bytes = base_image["image"]  # 圖片的二進制數據
        image_ext = base_image["ext"]  # 圖片的格式 (通常是 'png' 或 'jpeg')

        # 生成圖片檔名
        image_filename = os.path.join(output_folder, f"page{page_number+1}_img{img_index+1}.{image_ext}")

        # 保存圖片
        with open(image_filename, "wb") as image_file:
            image_file.write(image_bytes)

        image_count += 1
        print(f"圖片已儲存：{image_filename}")

    if image_count == 0:
        print("此頁沒有圖片。")

# 使用範例

extract_images_from_page(pdf_file, 27)


## Gemini測試

In [ ]:
from google import genai
from google.genai import types

api_key = input('input your api key')

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=[read_pdf_text(pdf_file, 25)],
                config = types.GenerateContentConfig(
                    max_output_tokens=1000,
                    temperature=0.5,
                    system_instruction="你的工作是理解這一頁論文的內容，並判斷文字描述的是實際研究相關的內容，還是大標題、目錄、相關文件等等無關緊要的內容。當內容是實際研究相關的內容，請回傳[response: 1]的格式，若是其他無關緊要的內容，請回傳[response: 0]"
                        )
                    )
print(response.text)